## 라이브러리 불러오기 ##

In [ ]:
import torchvision
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

from IPython.display import display
import ipywidgets
import ipywidgets.widgets as widgets
import traitlets
from jetbot import Robot, Camera, bgr8_to_jpeg

import paho.mqtt.client as mqtt
import threading
import time
from SCSCtrl import TTLServo

## 카메라, 로봇 객체 초기화하기 & road Detection 모델 불러오기

In [ ]:
## 모델 불러오기
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model.load_state_dict(torch.load('best_steering_model_xy_test.pth'))

device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()
print('model load success')

In [ ]:
#로봇 초기객체 초기화
robot = Robot()
#모터 초기화(Arm모터)
TTLServo.xyInputSmooth(100, 0, 1)
TTLServo.servoAngleCtrl(5, 50, 1, 100)
TTLServo.servoAngleCtrl(4, 10, 1, 100)
print('robot arm Ready!')

#카메라 객체 초기화
# MQTT 클라이언트 설정
broker_address = "127.0.0.1"
topic = "agv0/image"
subscribe_topic = "agv0/message"

client = mqtt.Client()
client.connect(broker_address)

# JetBot 카메라 설정
#camera = Camera.instance(width=300, height=300)
camera = Camera()
# 카메라로 찍은 frame을 띄워줄 image 객체 생성, 크기를 맞출 필요는 없다.
image = ipywidgets.Image(format='jpeg', width=300, height=300)
display(image)

# MQTT 전송 및 이미지 처리 함수
def execute(camera_image):
    frame = np.copy(camera_image)
    if frame is not None:
        img_str = cv2.imencode('.jpg', frame)[1].tobytes()
        client.publish(topic, img_str)
    return bgr8_to_jpeg(frame)

# MQTT 수신 메시지 처리 함수
def on_message(client, userdata, message):
    msg_payload = message.payload.decode()
    # 전방 물체 감지 확인후 수동 전환
    if msg_payload == "Object Detection" and startBtn.description == "Stop":
        print(msg_payload)
        start(True)
        
    # 모드 전환
    elif msg_payload == "mode_change":
        print(msg_payload)
        start(True)
        
    elif msg_payload == "red" and startBtn.description == "Start":
        print(msg_payload)
        change_color("red")
        start(True)

    elif msg_payload == "blue" and startBtn.description == "Start":
        print(msg_payload)
        change_color("blue")
        start(True)
        
    elif msg_payload == "orange" and startBtn.description == "Start":
        print(msg_payload)
        change_color("orange")
        start(True)
        
    elif msg_payload == "green" and startBtn.description == "Start":
        print(msg_payload)
        change_color("green")
        start(True)
        
    elif msg_payload == "purple" and startBtn.description == "Start":
        print(msg_payload)
        change_color("purple")
        start(True)
        
    elif msg_payload == "yellow" and startBtn.description == "Start":
        print(msg_payload)
        change_color("yellow")
        start(True)
    
client.on_message = on_message
client.subscribe(subscribe_topic)
client.loop_start()
traitlets.dlink((camera, 'value'), (image, 'value'), transform=execute)

## 위젯 추가 및 바인딩

In [ ]:
lbl10 = ipywidgets.Label(value="Center Max :")
colorlblMax = ipywidgets.Label(value="0")
lbl11 = ipywidgets.Label(value="areaA :")
areaAlbl = ipywidgets.Label(value="None")
hbox1 = widgets.HBox([lbl10, colorlblMax, lbl11, areaAlbl])

lbl21 = ipywidgets.Label(value="Center Min :")
colorlblMin = ipywidgets.Label(value="0")
lbl22 = ipywidgets.Label(value="areaB :")
areaBlbl = ipywidgets.Label(value="None")
hbox2 = widgets.HBox([lbl21, colorlblMin,lbl22,areaBlbl])

lbl30 = ipywidgets.Label(value="Center Color :")
colorlbl = ipywidgets.Label(value="None")
lbl31 = ipywidgets.Label(value="self.flag :")
flaglbl = ipywidgets.Label(value="None")
hbox3 = widgets.HBox([lbl30, colorlbl,lbl31, flaglbl])
vbox1 = widgets.VBox([hbox1, hbox2, hbox3])

image_widget = ipywidgets.Image(format='jpeg', width=224, height=224)
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')
vbox2 = widgets.VBox([image_widget, x_slider, steering_slider], layout=widgets.Layout(align_self='center'))
hbox4 = widgets.HBox([vbox2, y_slider, speed_slider], layout=widgets.Layout(align_self='center'))

startBtn = widgets.Button(description="Start", button_style='info')
lbl41 = ipywidgets.Label(value="Find Area : ")
goallbl = ipywidgets.Label(value="None")
hbox5 = widgets.HBox([startBtn,lbl41, goallbl])

#수동 조작용 contorller widget 생성
lbl50 = ipywidgets.Label(value="Manual Controller")
modeBtn = widgets.Button(description="Auto", button_style='success')
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)
middle_box = widgets.HBox([left_button, stop_button, right_button], layout=widgets.Layout(align_self='center'))
controls_box = widgets.VBox([lbl50,modeBtn, forward_button, middle_box, backward_button])

#자동 조작용 contorller widget 생성
lbl51 = ipywidgets.Label(value="Auto Controller")
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.30, description='speed gain',disabled = False)
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.22, description='steering gain',disabled = False)
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd',disabled = False)
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias',disabled = False)
vbox3 = widgets.VBox([lbl51,speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider])
hbox6 = widgets.HBox([controls_box, vbox3], layout=widgets.Layout(align_self='center'))

display(vbox1,hbox4,hbox5,hbox6)

manual_btnlst = [stop_button, forward_button, backward_button, left_button, right_button]
camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

## 색깔인식 범위 지정

In [18]:
#set areaA
areaA = 'red'
colors = [
        {'name': 'red', 'lower': np.array([0, 80, 140]), 'upper': np.array([10, 180, 200])},
        {'name': 'green', 'lower': np.array([50, 70, 120]), 'upper': np.array([80,180, 200])},
        {'name': 'blue', 'lower': np.array([80, 50, 50]), 'upper': np.array([110, 250, 200])},
        {'name': 'purple', 'lower': np.array([105, 60, 110]), 'upper': np.array([130, 165, 154])},
        {'name': 'yellow', 'lower': np.array([25, 90, 160]), 'upper': np.array([30, 200, 210])},
        {'name': 'orange', 'lower': np.array([10, 100, 140]), 'upper': np.array([20, 180, 200])},
]

areaA_color = next((color for color in colors if color['name'] == areaA), None)
areaAlbl.value = areaA_color['name']

findArea = areaA
goallbl.value = findArea

#frame 크기와 카메라 중심점 좌표 설정
frame_width = 120
frame_height = 320
camera_center_X = int(frame_width/2)
camera_center_Y = int(frame_height/2)

#AreaFind() 용 리스트와 변수
colorHSVvalueList = []
max_len = 20

def change_color(color):
    global areaA, areaA_color
    areaA = color

    areaA_color = next((color for color in colors if color['name'] == areaA), None)
    areaAlbl.value = areaA_color['name']

    findArea = areaA
    goallbl.value = findArea


#2개 thread 용 객체 변수 생성
roadFinding = None
goalFinding = None

## Controller변수 설정

In [ ]:
#joystick controller 용 객체 변수생성
ctrlArm = None
inputThreading = None

moveStartTor = 0.15    # 모터의 초기 회전 임계값입니다.
fb_input = 0           # 전진 및 후진 방향의 속도 매개변수를 저장합니다.
lr_input = 0           # 조종을 위한 매개변수를 저장합니다.
servoCtrlTime = 0.001  # 오류 방지를 위해 TTL 서보 통신 후 적용되는 지연시간
speedInput = 300       # 카메라 기울기 및 팬 회전 속도.
armXStatus = 0         # 로봇 팔의 X축 이동 상태(원거리 및 근거리 방향)입니다.
armYStatus = 0         # 로봇팔의 Y축(수직방향)의 이동상태.
goalX = 130            # 로봇팔 그리퍼의 X축 좌표를 저장합니다.
goalY = 50             # 로봇팔 그리퍼의 Y축 좌표를 저장합니다.
xMax = 210             # 로봇팔의 최대 X축을 설정합니다.
xMin = 100             # 로봇팔의 최소 X축을 설정합니다.
yMax = 120             # 로봇팔의 최대 Y축을 설정합니다.
yMix = -200            # 로봇팔의 최소 Y축을 설정합니다.
movingTime = 0.005     # 로봇 팔의 역운동학 기능의 인접한 두 위치 사이의 실행 시간을 설정합니다
movingSpeed = 1        # 로봇팔의 이동속도를 설정합니다.
grabStatus = 0         # 그리퍼의 움직임 상태입니다.

import os, struct, array
from fcntl import ioctl

# Iterate over the joystick devices.
print('Available devices:')
for fn in os.listdir('/dev/input'):
    if fn.startswith('js'):
        print('  /dev/input/%s' % (fn))

# Linux 시스템의 조이스틱/게임패드 장치(/dev/input/js0)에서 입력을 읽은 다음 로봇이나 유사한 장치를 제어한다.
# /dev/input 'js'로 시작하는 장치들이 있다면 print()
# js0, js1 게임패드나 조이스틱이 해당됨.

# These constants were borrowed from linux/input.h
axis_names = {
    0x00 : 'x',
    0x01 : 'y',
    0x02 : 'z',
    0x03 : 'rx',
    0x04 : 'ry',
    0x05 : 'rz',
    0x06 : 'trottle',
    0x07 : 'rudder',
    0x08 : 'wheel',
    0x09 : 'gas',
    0x0a : 'brake',
    0x10 : 'lr',
    0x11 : 'fb',
    0x12 : 'hat1x',
    0x13 : 'hat1y',
    0x14 : 'hat2x',
    0x15 : 'hat2y',
    0x16 : 'hat3x',
    0x17 : 'hat3y',
    0x18 : 'pressure',
    0x19 : 'distance',
    0x1a : 'tilt_x',
    0x1b : 'tilt_y',
    0x1c : 'tool_width',
    0x20 : 'volume',
    0x28 : 'misc',
}

button_names = {
    0x120 : 'trigger',
    0x121 : 'thumb',
    0x122 : 'thumb2',
    0x123 : 'top',
    0x124 : 'top2',
    0x125 : 'pinkie',
    0x126 : 'base',
    0x127 : 'base2',
    0x128 : 'base3',
    0x129 : 'base4',
    0x12a : 'base5',
    0x12b : 'base6',
    0x12f : 'dead',
    0x130 : 'a',
    0x131 : 'b',
    0x132 : 'c',
    0x133 : 'x',
    0x134 : 'y',
    0x135 : 'z',
    0x136 : 'tl',
    0x137 : 'tr',
    0x138 : 'tl2',
    0x139 : 'tr2',
    0x13a : 'select',
    0x13b : 'start',
    0x13c : 'mode',
    0x13d : 'thumbl',
    0x13e : 'thumbr',

    0x220 : 'dpad_up',
    0x221 : 'dpad_down',
    0x222 : 'dpad_left',
    0x223 : 'dpad_right',

    0x2c0 : 'dpad_left',
    0x2c1 : 'dpad_right',
    0x2c2 : 'dpad_up',
    0x2c3 : 'dpad_down',
}
## 축과 버튼의 이름을 알기위한 키 맵핑을 위해 사용
axis_map = []
button_map = []

# Open the joystick device.
# 장치 열기

fn = '/dev/input/js0'
print('Opening %s...' % fn)
jsdev = open(fn, 'rb')

# Get the device name.
buf = array.array('B', [0] * 64)
# 바이트(unsigned char)형으로 크기가 64인 array 생성
ioctl(jsdev, 0x80006a13 + (0x10000 * len(buf)), buf) # JSIOCGNAME(len)
# 조이스틱 이름 가져오기
js_name = buf.tobytes().rstrip(b'\x00').decode('utf-8')
print('Device name: %s' % js_name)

# Get number of axes and buttons.

buf = array.array('B', [0])
ioctl(jsdev, 0x80016a11, buf)
num_axes = buf[0]
# 축 갯수 가져오기 buf[0] 에 저장되어 있음.

buf = array.array('B', [0])
ioctl(jsdev, 0x80016a12, buf)
num_buttons = buf[0]
# 버튼 갯수 가져오기 buf[0] 에 저장되어 있음.
# Get the joystick address and correspond to the name in axis_names.
# 조이스틱의 주소와 축 이름에서 해당되는 이름 가져오기
buf = array.array('B', [0] * 0x40)
ioctl(jsdev, 0x80406a32, buf) # JSIOCGAXMAP
# 총 축의 수만큼 loop돌림
for axis in buf[:num_axes]:
    axis_name = axis_names.get(axis, 'unknown(0x%02x)' % axis)
    #axis_names의 첫번째 번호와 같은 이름을 가져온다.
    #ex) axis가 0이면 axis_names에서 0x00 > 'x' 를 가져온다.
    #ex) axis가 1이면 axis_names에서 0x01 > 'y' 를 가져온다.
    axis_map.append(axis_name)
    # 해당 축 초기화

# Get the button address and correspond to the name in button_names.
# 버튼 번호로 이름을 가져온다.
buf = array.array('H', [0] * 200)
ioctl(jsdev, 0x80406a34, buf) # JSIOCGBTNMAP

for btn in buf[:num_buttons]:
    btn_name = button_names.get(btn, 'unknown(0x%03x)' % btn)
    button_map.append(btn_name)

print('%d axes found: %s' % (num_axes, ', '.join(axis_map)))
print('%d buttons found: %s' % (num_buttons, ', '.join(button_map)))


## Joystick 용 함수 정의하기

In [20]:
# The robotic arm stops the current action.
def armStop():
    global armXStatus, armYStatus
    if armXStatus or armYStatus:
        armXStatus = 0
        armYStatus = 0
        TTLServo.servoStop(2)
        time.sleep(servoCtrlTime)
        TTLServo.servoStop(3)
        time.sleep(servoCtrlTime)
        print([goalX, goalY])

# Control the gripping and loosening of the gripper.
def grabCtrlCommand(commandType, valueInput):
    global grabStatus
    if commandType == 'tl':
        commandLoose = 0
        if valueInput:
            commandGrab = 1
        else:
            commandGrab = 0
    elif commandType == 'tl2':
        commandGrab = 0
        if valueInput:
            commandLoose = 1
        else:
            commandLoose = 0
    else:
        return

    if commandGrab and not grabStatus:
        TTLServo.servoAngleCtrl(4, -90, 1, 150)
        grabStatus = 1
        time.sleep(servoCtrlTime)
    elif commandLoose and not grabStatus:
        TTLServo.servoAngleCtrl(4, 90, 1, 150)
        grabStatus = 1
        time.sleep(servoCtrlTime)

    # Grab 명령을 처리중일 때 Grap상태가 아니라면
    # Grab 형태로 서보모터 이동
    # Loose 명령을 처리중일 때 Grap상태가 아니라면
    # Loose 형태로 서보모터 이동
    
    if not commandGrab and not commandLoose and grabStatus:
        TTLServo.servoStop(4)
        grabStatus = 0
        time.sleep(servoCtrlTime)
    # 처리될 명령이 없고, grapStatus가 1일 때 grab(4번 서보모터) 멈춤 

# Robotic arm control function, for command input.
def armCtrlCommand(commandType, valueInput):
    global xMax, xMin, yMax, yMix, goalX, goalY, armXStatus, armYStatus
    commandXin, commandXde, commandYin, commandYde = 0, 0, 0, 0
    if commandType == 'y':
        if valueInput:
            ctrlArm.moveXin = 1
            print('run')
        else:
            ctrlArm.moveXin = 0
    # 'y' 넘겨 받았을 때, value input받으면 x 축 방향 증가
    # 아니면 x 축방향으로 증가하지 않음.
    elif commandType == 'a':
        if valueInput:
            ctrlArm.moveXde = 1
        else:
            ctrlArm.moveXde = 0
    # 'a' 넘겨 받았을 때, value input받으면 x 축 방향 감소
    # 아니면 x 축방향으로 감소하지 않음.
    elif commandType == 'tr':
        if valueInput:
            ctrlArm.moveYin = 1
        else:
            ctrlArm.moveYin = 0
    # 'tr' 넘겨 받았을 때, value input받으면 y 축 방향 증가
    # 아니면 y 축방향으로 증가하지 않음.
    elif commandType == 'tr2':
        if valueInput:
            ctrlArm.moveYde = 1
        else:
            ctrlArm.moveYde = 0
    else:
        return
    # 'tr2' 넘겨 받았을 때, value input받으면 y 축 방향 감소
    # 아니면 y 축방향으로 감소하지 않음.
    
# Robotic arm control function, for action execution (in another single thread).
def armCtrl(commandXin, commandXde, commandYin, commandYde):
    global xMax, xMin, yMax, yMix, goalX, goalY, armXStatus, armYStatus
    if commandXin:
        armXStatus = 1
        goalX += movingSpeed
        if goalX > xMax:
            goalX = xMax
        TTLServo.xyInputSmooth(goalX, goalY, movingTime)
        time.sleep(servoCtrlTime)
        return
    # commandXin 일 때, goalX(로봇팔 그리퍼의 X축 좌표) movingSpeed를 더해 모터를 움직임.
    # 로봇팔의 최대 X축을 넘지 못하도록 설정
    elif commandXde:
        armXStatus = 1
        goalX -= movingSpeed
        if goalX < xMin:
            goalX = xMin
        TTLServo.xyInputSmooth(goalX, goalY, movingTime)
        time.sleep(servoCtrlTime)
        return
    # commandXde 일 때, goalX(로봇팔 그리퍼의 X축 좌표) movingSpeed를 빼서 모터를 움직임.
    # 로봇팔의 최소 X축을 넘지 못하도록 설정
    if commandYin:
        armYStatus = 1
        goalY += movingSpeed
        if goalY > yMax:
            goalY = yMax
        TTLServo.xyInputSmooth(goalX, goalY, movingTime)
        time.sleep(servoCtrlTime)
        return
    # commandYin 일 때, goalY(로봇팔 그리퍼의 Y축 좌표) movingSpeed를 더해 모터를 움직임.
    # 로봇팔의 최대 Y축을 넘지 못하도록 설정
    elif commandYde:
        armYStatus = 1
        goalY -= movingSpeed
        if goalY < yMix:
            goalY = yMix
        TTLServo.xyInputSmooth(goalX, goalY, movingTime)
        time.sleep(servoCtrlTime)
        return
    # commandYde 일 때, goalY(로봇팔 그리퍼의 Y축 좌표) movingSpeed를 빼서 모터를 움직임.
    # 로봇팔의 최소 Y축을 넘지 못하도록 설정
    if not commandXin and not commandXde and not commandYin and not commandYde:
        armStop()
    # 로봇팔의 x,y축에 대한 이동 관련 명령이 없는 경우,
    # armStop() 호출

# Robot movement stops.
def moveStop():
    global fb_input, lr_input
    fb_input = 0
    lr_input = 0
    robot.stop()

# Robot movement control.
def moveSmoothCtrl(commandType, valueInput):
    global fb_input, lr_input
    if commandType == 'y':
        fb_input = valueInput
    elif commandType == 'x':
        lr_input = valueInput
    else:
        return
    # commandType에 따라 valueInput을 fb_input(앞으로/뒤로 입력) 또는 lr_input(왼쪽/오른쪽 입력)에 할당
    # commandType이 'y' 또는 'x'와 일치하지 않으면 함수는 추가 작업 없이 반환
    if abs(fb_input) < moveStartTor and abs(lr_input) < moveStartTor:
        moveStop()
        return
    # fb_input과 lr_input의 절대값이
    # 모터의 회전 임계값을 넘지 않으면 moveStop() 함수 호출
    
    # fb_input 또는 lr_input의 절대값이 moveStartTor보다 높으면 
    # leftSpeed 및 rightSpeed를 계산.
    leftSpeed  = -fb_input + lr_input
    rightSpeed = -fb_input - lr_input
    robot.left_motor.value  = leftSpeed
    robot.right_motor.value = rightSpeed
    # 차동 조향을 위한 두 개의 모터(왼쪽 및 오른쪽 모터)를 사용하여 
    # 로봇의 움직임을 제어한다.

## Joystick 용 스레드 생성하기

In [21]:
class ArmCtrlThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(ArmCtrlThread, self).__init__(*args, **kwargs)
        self.moveXin = 0
        self.moveXde = 0
        self.moveYin = 0
        self.moveYde = 0
        self.th_flag = True
        print("수동 암")
        
    def st(self):
        self.th_flag = True

    def stop(self):
        self.th_flag = False

    def run(self):
        while self.th_flag:
            
            armCtrl(self.moveXin, self.moveXde, self.moveYin, self.moveYde)
            time.sleep(movingTime)
        robot.stop()
        print("ArmCtrlThread End")


class inputThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(inputThread, self).__init__(*args, **kwargs)
        self.th_flag = True
        print("수동 이동")
    
    def st(self):
        self.th_flag = True

    def stop(self):
        self.th_flag = False

    def run(self):
        
        while self.th_flag:
            evbuf = jsdev.read(8)
            if evbuf:
                time, value, type, number = struct.unpack('IhBB', evbuf)
                if type & 0x80:
                    print("(initial)", end="")
                if type & 0x01:
                    button = button_map[number]
                    if button:
                        grabCtrlCommand(button, value)
                        armCtrlCommand(button, value)
                if type & 0x02:
                    axis = axis_map[number]
                    if axis:
                        moveSmoothCtrl(axis, value / 32767.0)
        robot.stop()
        print("inputThread End")

## WorkingAreaFind() 클래스 생성하기 / 색깔 인식

In [22]:
class WorkingAreaFind(threading.Thread):
    flag = 1
    def __init__(self):
        super().__init__()
        self.th_flag=True
        self.imageInput = 0
        flaglbl.value = str(WorkingAreaFind.flag)
        self._stop_event = threading.Event()
        
    def st(self):
        self.th_flag = True

    def run(self):
        while self.th_flag:
            self.imageInput = camera.value
            #BGR to HSV
            hsv = cv2.cvtColor(self.imageInput, cv2.COLOR_BGR2HSV)
            #blur
            hsv = cv2.blur(hsv, (15, 15))
            self.colorRecog(hsv)

            #areaA, areaB Color searching
            areaA_mask = cv2.inRange(hsv, areaA_color['lower'], areaA_color['upper'])
            areaA_mask = cv2.erode(areaA_mask, None, iterations=2)
            areaA_mask = cv2.dilate(areaA_mask, None, iterations=2)


            # 해당 영역에 대한 윤곽선 따기
            AContours, _ = cv2.findContours(areaA_mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            #A영역
            if AContours and WorkingAreaFind.flag == 1:
                self.findCenter(areaA, AContours)

            #두 영역 모두 못찾았다면, 찾아가는 중이다.
            else:
                cv2.putText(self.imageInput, "Finding...", (0, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1,cv2.LINE_AA)
                image_widget.value = bgr8_to_jpeg(self.imageInput)
            time.sleep(0.1)
            

    #name : A, B 구분용도, Contours 각 영역의 윤곽선 값
    def findCenter(self, name, Contours):
        c = max(Contours, key=cv2.contourArea)
        ((box_x, box_y), radius) = cv2.minEnclosingCircle(c)

        X = int(box_x)
        Y = int(box_y)

        error_Y = abs(camera_center_Y - Y)
        error_X = abs(camera_center_X - X)

        if error_Y < 15 and error_X < 15:
            #A영역이 가까이 오게됨
            if name == areaA and self.flag == 1:
                robot.stop()
                #test 코드
                areaAlbl.value = areaA + " Goal!"
            

                ###########################
                print("start_go")
                start(True)
               
                ###########################
                print("Road Following End")

                
        image_widget.value = bgr8_to_jpeg(self.imageInput)

    def colorRecog(self, hsv):
        # Center Pixel 의 hsv 값 읽어오기
        hsvValue = hsv[int(frame_height / 2), int(frame_width / 2)]
            
        # data 20개 모아서, 최대, 최소 값 구하기
        colorHSVvalueList.append(hsvValue)
        if len(colorHSVvalueList) > max_len:
            del colorHSVvalueList[0]
                
        max_h, max_s, max_v = np.maximum.reduce(colorHSVvalueList)
        min_h, min_s, min_v = np.minimum.reduce(colorHSVvalueList)
        # Center Pixel 주위에 20x20 크기의 사각형 그리기
        rect_s = 20
        cv2.rectangle(self.imageInput,
                    (int(frame_width / 2) - int(rect_s / 2), int(frame_height / 2) - int(rect_s / 2)),
                    (int(frame_width / 2) + int(rect_s / 2), int(frame_height / 2) + int(rect_s / 2)),
                    (0, 0, 255), 1)
            
        # max, min value 표시
        cv2.putText(self.imageInput, f'max_HSV:{ max_h, max_s, max_v }', (20, 180), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(self.imageInput, f'min_HSV:{ min_h, min_s, min_v }', (20, 200), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        colorlblMax.value = str(max_h) + "," + str(max_s) + "," + str(max_v)
        colorlblMin.value = str(min_h) + "," + str(min_s) + "," + str(min_v) 
              

    def stop(self):
        self.th_flag = False
        self._stop_event.set()
        robot.stop()

목적지 인자를 받아서 자동이동하는 스레드

## RobotMoving() 클래스 생성하기 / road detection


In [23]:
class RobotMoving(threading.Thread):
    print('robot moving!')
    def __init__(self):
        super().__init__()
        self.th_flag = True
        self.angle = 0.0
        self.angle_last = 0.0
        self._stop_event = threading.Event()
        
    def st(self):
        self.th_flag = True

    def run(self):
        while self.th_flag:
            image = camera.value
            xy = model(self.preprocess(image)).detach().float().cpu().numpy().flatten()
            x = xy[0]
            y = (0.5 - xy[1]) / 2.0

            x_slider.value = x
            y_slider.value = y

            speed_slider.value = speed_gain_slider.value
            image_widget.value = bgr8_to_jpeg(image)

            self.angle = np.arctan2(x, y)

            if not self.th_flag:
                break

            pid = self.angle * steering_gain_slider.value + (self.angle - self.angle_last) * steering_dgain_slider.value
            self.angle_last = self.angle

            steering_slider.value = pid + steering_bias_slider.value

            robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
            robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
            time.sleep(0.1)
        robot.stop()
        print("Area Finding End")

    def preprocess(self, image):
        image = PIL.Image.fromarray(image)
        image = transforms.functional.to_tensor(image).to(device).half()
        image.sub_(mean[:, None, None]).div_(std[:, None, None])
        return image[None, ...]

    def stop(self):
        self.th_flag = False
        self._stop_event.set()
        robot.stop()

robot moving!


## Start 버튼 함수 정의하고 바인딩하기

In [24]:
# 시작하자마자 기본 수동상태 전환
ctrlArm = ArmCtrlThread()
ctrlArm.start()
inputThreading = inputThread()
inputThreading.start()
#roadFinding = RobotMoving()
#goalFinding = WorkingAreaFind()


def start(change):
    global modeBtn, camera_link, goalFinding, roadFinding, areaA, findArea, goallbl, ctrlArm, inputThreading
    #Start -> Auto
    if startBtn.description == "Start":
        modeBtn.value = "Auto"
        modeBtn.disabled = True

        for i in manual_btnlst:
            i.disabled = True

        goalFinding = WorkingAreaFind()
        goalFinding.start()
        roadFinding = RobotMoving()
        roadFinding.start()

        startBtn.button_style = "warning"
        startBtn.description = "Stop"
        
        # 기존 스레드 종료
        ctrlArm.stop()
        #ctrlArm.join()
        inputThreading.stop()
        #inputThreadinga.join()
        # 카메라 링크 해제
        camera_link.unlink()

    elif startBtn.description == "Stop":
        roadFinding.stop()
        goalFinding.stop()
        print("Warning!!")

        ctrlArm = ArmCtrlThread()
        ctrlArm.start()
        inputThreading = inputThread()
        inputThreading.start()

        startBtn.button_style = "info"
        startBtn.description = "Start"
        for i in manual_btnlst:
            i.disabled = False
        modeBtn.value = "Auto"
        modeBtn.disabled = False
        camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
        
startBtn.on_click(start)

수동 암
수동 이동
(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)(initial)

## 프로젝트 종료하기
로봇팔 초기화, 스레드 종료, 카메라/로봇 객체 제거

In [27]:
# 로봇팔 초기화하기
TTLServo.xyInputSmooth(100, 0, 1)
TTLServo.servoAngleCtrl(5, 50, 1, 100)
TTLServo.servoAngleCtrl(4, 10, 1, 100)

client.disconnect()
time.sleep(0.1)
robot.stop()
camera.stop()
print('End')

End
